# 5. 循环神经网络RNN

循环神经网络（RNN）由于其特有的循环结构，非常适合处理序列型数据，RNN的参数对于整个序列数据共享，因此它可以从数据中提取时序信息。由于RNN的时序处理能力，它常被用于翻译、语音识别等自然语言处理领域。

## 5.1 RNN

### 5.1.1 RNN的结构	

RNN也属于神经网络，基本结构包含输入层、输出层、隐藏层，不同的是它包含一个循环结构：

<img src="./imgs/图片1.png" alt="image-20210710104037967" style="zoom:80%;" />

​	如果我们将RNN展开，就得到了下图，可以看到每次有了新的输入x，隐藏层的信息A都会把当前状态向下一个状态传播，这也是RNN包含序列信息的原因：

<img src="./imgs/图片2.png" alt="image-20210710104200532" style="zoom:80%;" />

​	我们接着观察RNN的内部结构：

<img src="./imgs/图片3.png" alt="image-20210710104523070" style="zoom:80%;" />

​	RNN的参数包含3部分$U:输入到隐藏层, V:隐藏层到输出层, W:隐藏层到隐藏层$，每个新的输入与$U$相乘，上个状态的隐藏层与$W$相乘，二者相加得到当前状态的隐藏层，隐藏层输出一方面经过激活函数，与$V$相乘再到输出层，另一方面向下一个状态传播。

### 5.1.2 RNN的前向传播与反向传播算法

前向传播：

<img src="./imgs/图片4.png" alt="image-20210710105400694" style="zoom:67%;" />

反向传播：**BPTT(Back Prropagation Through TIme)**

<img src="./imgs/图片5.png" alt="image-20210710105443302" style="zoom:67%;" />

## 5.2 LSTM

### 5.2.1 RNN存在的问题：长依赖关系Long-Term Dependencies

RNN可以将以前的信息连接到当前的任务中，但它并不总是有效的。由于RNN的链式结构，随着图中间隔的增大，以前的信息将会不断减弱，也就是说网络会逐渐遗忘以前的信息。例如，“I grew up in France… I speak fluent *French*”，显然France和French有很强的关联关系，但由于其间隔较远，网络将会逐渐遗忘France，而fluent将会是影响下一个单词最重要的因素。

<img src="./imgs/图片6.png" alt="image-20210710105913168" style="zoom:80%;" />

### 5.2.2 LSTM的原理

RNN会将每个状态的信息全部保存下来传播到下一状态，而LSTM引入了门 Gate的概念，门可以允许网络去选择保留什么信息。如下图Memory Cell是中间记忆，每次输入首先通过输入门选择存储，当储存到Memory Cell后，网络会通过遗忘门选择性保留有用的信息。

<img src="./imgs/图片7.png" alt="image-20210710110433529" style="zoom:80%;" />

### 5.2.3 LSTM的结构

相比RNN，LSTM有更复杂的循环结构：

<img src="./imgs/图片8.png" alt="image-20210710111042387" style="zoom:70%;" />

<img src="./imgs/图片9.png" alt="image-20210710111125350" style="zoom:67%;" />

LSTM的核心是细胞状态Cell State，它包含了之前状态的信息，是网络的记忆：

<img src="./imgs/图片10.png" alt="image-20210710111311470" style="zoom:67%;" />

LSTM通过门来选择哪些信息继续传播，通常门是一个sigmoid网络层，注意sigmoid的输出在01之间，因此通过将门与细胞状态相乘，有用的信息被保留，而无用信息与很小的数相乘，被消除。

<img src="./imgs/图片11.png" alt="image-20210710111718651" style="zoom:67%;" />

### 5.2.4 LSTM的前向传播过程

首先网络根据上一个状态的信息和新信息计算遗忘门，遗忘门也是一个sigmoid网络层，当遗忘门与上一个细胞状态相乘，就选择性保留了以前的信息：

<img src="./imgs/图片12.png" alt="image-20210710112037809" style="zoom:67%;" />

接着要选择保留当前状态的信息，包括两个部分。首先输入门，一个sigmoid层，选择哪些值被更新，另一个tanh层用于生成更新细胞，二者相乘就保留所需信息，再与以前的信息相加就完成了当前的细胞状态。

<img src="./imgs/图片13.png" alt="image-20210710112722624" style="zoom:67%;" />

所以细胞状态的更新包括：与遗忘门相乘，再与更新细胞相加。

<img src="./imgs/图片14.png" alt="image-20210710112817019" style="zoom:67%;" />

输出时细胞状态先通过一个tanh门，被压缩至-1到1之间，再与通过sigmoid门的输入相乘：

<img src="./imgs/图片15.png" alt="image-20210710113225392" style="zoom:67%;" />

## 5.3 GRU

GRU是LSTM的一种变体，它将遗忘门和输入门合成为更新门，参数相比LSTM更少，但准确率相近，过拟合的可能更小。

<img src="./imgs/图片16.png" alt="image-20210710113607169" style="zoom:67%;" />

## 5.4 使用LSTM完成推特情感分析  
本部分使用Keras实现推特数据集情感分类任务。  
数据集：nltk的推特情感分析语料库，所有推特分为开心1和不开心0两类，共10000条数据。  
环境：keras，keras里面有Tokenizer等包可以很方便地处理文字数据，模型实现也比较简单。

### 5.4.1 导入需要的包

In [ ]:
# 导入包
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords, twitter_samples
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

### 5.4.2 下载语料库  
twitter_samples是推特数据集，每条推特的标签为0不开心，1开心  

stopwords是停词数据，停词是对语义无大影响的单词，比如is，a

In [3]:
# 下载语料库
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

###  5.4.3 划分训练集和测试集

In [4]:
# 把推特分为训练集和测试集
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]
train_x = train_pos + train_neg
test_x = test_pos + test_neg

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [12]:
# 查看部分推特
train_pos[:5]

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days']

In [13]:
train_neg[:5]

['hopeless for tmr :(',
 "Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(",
 '@Hegelbon That heart sliding into the waste basket. :(',
 '“@ketchBurning: I hate Japanese call him "bani" :( :(”\n\nMe too',
 'Dang starting next week I have "work" :(']

### 5.4.4 去除停词和stemming  
去除停词可以使句子在不影响语义的情况下变得更短；  

stemming是在英语中将同一单词的不同形式，如第三人称等都进行统一。

In [5]:
# 去除停词和stemming
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
def preprocess(text, stem=False):
  
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

df = pd.DataFrame({'data':train_x+test_x, 'label':np.concatenate((train_y,test_y))})
df.data = df.data.apply(lambda x: preprocess(x,True))
df.head()

,data,label
0,followfriday franc int pkuchly57 milipol pari ...,1.0
1,hey jame odd pleas call contact centr 02392441...,1.0
2,listen last night bleed amaz track scotland,1.0
3,congrat,1.0
4,yeaaaah yippppi accnt verifi rqst succeed got ...,1.0


### 5.4.5 Tokenizer  
Tokenizer首先将文本数据转化为一个单词和频率的词典，如I 10000， am 500;  

然后根据字典的下标，Tokenizer可以将文本的句子转化为向量，如 I love you->[2 50 3]

In [6]:
# tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_x+test_x)
vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

SEQUENCE_LENGTH = 200
x_train = pad_sequences(tokenizer.texts_to_sequences(df.data.iloc[:8000]), maxlen=SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(df.data.iloc[8000:]), maxlen=SEQUENCE_LENGTH)

Total words 21573


### 5.4.6 定义模型  
我们的模型包括一个Embedding层，可以将数据降维至所需长度；  
一层LSTM层；  
一层全连接层用于二分类。

In [7]:
# 定义模型：LSTM功能比较强大，数据集较小就只用了一个LSTM层
maxlen = 21573
embed_dim = 200

model = Sequential()
model.add(Embedding(maxlen, embed_dim, input_length = 200)) # Embedding数据降维
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) # LSTM
model.add(Dense(1,activation='sigmoid')) # 全连接 2分类
model.compile("adam", "binary_crossentropy", metrics=["accuracy"]) #binary_crossentropy
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 200)          4314600   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               120400    
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 4,435,101
Trainable params: 4,435,101
Non-trainable params: 0
_________________________________________________________________
None


### 5.4.7 结果
训练了10个epoch，训练集准确率为94.3%，测试集准确率为66.8%，有一定的过拟合。

In [8]:
model.fit(x_train, train_y, batch_size=64, epochs=10, validation_split=0.1)

Epoch 1/10
113/113 [==============================] - 104s 727ms/step - loss: 0.6570 - accuracy: 0.5780 - val_loss: 0.6833 - val_accuracy: 0.6175
Epoch 2/10
113/113 [==============================] - 81s 717ms/step - loss: 0.4132 - accuracy: 0.8129 - val_loss: 0.6823 - val_accuracy: 0.6712
Epoch 3/10
113/113 [==============================] - 83s 731ms/step - loss: 0.3171 - accuracy: 0.8608 - val_loss: 0.8245 - val_accuracy: 0.6313
Epoch 4/10
113/113 [==============================] - 81s 719ms/step - loss: 0.2425 - accuracy: 0.8953 - val_loss: 0.9117 - val_accuracy: 0.6513
Epoch 5/10
113/113 [==============================] - 81s 715ms/step - loss: 0.2035 - accuracy: 0.9139 - val_loss: 0.8578 - val_accuracy: 0.6837
Epoch 6/10
113/113 [==============================] - 81s 715ms/step - loss: 0.1869 - accuracy: 0.9183 - val_loss: 1.0863 - val_accuracy: 0.6450
Epoch 7/10
113/113 [==============================] - 82s 723ms/step - loss: 0.1587 - accuracy: 0.9311 - val_loss: 1.3137 - val_a

In [9]:
score = model.evaluate(x_test, test_y, batch_size=64)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

32/32 [==============================] - 2s 50ms/step - loss: 1.2070 - accuracy: 0.6675

ACCURACY: 0.6675000190734863
LOSS: 1.206989049911499
